# Classification: Now with Text :D

Kaggle StumbleUpon Competition

https://www.kaggle.com/c/stumbleupon

** Competition **: 
1. Some web pages, such as news articles or seasonal recipes, are only relevant for a short period of time. Others continue to be important for a long time.
2. The goal is to identify pages which pages will be relevant for a short span of time, and which will be relevant for a long span on time and are thus considered "evergreen". 

** Evaluation **: Area under the curve (AUC) 


Import Python Modules 
=================

In [2]:
# quick hack to fix import path
# import sys; sys.path.append('/Users/julianalverio/code/conda/envs/sac/lib/python3.6/site-packages/')

# data manipulation
import pandas as pd
import numpy as np

# plots
%matplotlib inline
import random
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl

# classification algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

# dimensionality reduction
from sklearn.decomposition import PCA

# cross-validation
from sklearn.model_selection import train_test_split
from sklearn import model_selection

# text features
import re
from nltk import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# model evaluation
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir(os.path.join("..", "data"))

FileNotFoundError: [Errno 2] No such file or directory: '../data'

In [ ]:
os.getcwd()

# 1. Data Import

In [21]:
! wget -O home/ubuntu/machine_learning_aws/data/ "https://www.dropbox.com/s/10ch2yhfk8tyfri/train.tsv?dl=0" 

home/ubuntu/machine_learning_aws/data/: No such file or directory


In [12]:
import pandas as pd

#data = pd.read_table("stumbleupon/train.tsv", sep= "\t")
#data = pd.read_table("https://www.dropbox.com/s/10ch2yhfk8tyfri/train.tsv", sep= "\t")
data = pd.read_table("train.tsv?dl=0", sep= "\t")

In [14]:
data

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,1,14,0,12032,162,10,0.098765,0.082569,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,http://techcrunch.com/2010/09/08/kno-raises-46...,8958,"{""title"":""Kno Raises 46 Million More To Build ...",computer_internet,0.651067,3.010526,0.474747,0.222222,0.191919,0.191919,...,1,1,38,0,2219,99,11,0.040404,0.071429,0
7391,http://www.uncoached.com/category/why-i-miss-c...,8895,"{""title"":""Why I Miss College "",""body"":""Mar 30 ...",culture_politics,0.14192,2.208054,0.483333,0.246667,0.036667,0.026667,...,1,1,34,0,5672,300,4,0.020000,0.109453,0
7392,http://eatthis.menshealth.com/slide/sweet-pota...,1191,"{""title"":""Sweet Potatoes Eat This Not That i'...",recreation,0.196273,2.000000,0.315789,0.171053,0.105263,0.052632,...,?,1,43,0,848,76,5,0.434211,0.117647,1
7393,http://naturallyella.com/,5612,"{""title"":""Naturally Ella "",""body"":"" "",""url"":""n...",arts_entertainment,0.617876,1.026316,0.210526,0.052632,0.000000,0.000000,...,1,0,37,1,386,38,0,0.026316,0.333333,1


## Using Numerical Features (same as last week)

In [15]:
# Alchemy category, converting to one-hots
df = data['alchemy_category']   # 2K ? values
one_hots = pd.get_dummies(data['alchemy_category'])
df = one_hots
rename_dict = {'?': 'alchemy_cat_?'}
df = df.rename(columns=rename_dict)

# FrameTagRatio, leaving as continuous number
df_var = data['frameTagRatio']
df['frame_tag_ratio'] = df_var



# link word score, 0-100 gaussian, keeping continuous
df['link_word_score'] = data['linkwordscore']


# alchemy category score, with replacing missing values with random
df_var = data['alchemy_category_score']
df_var_temp = df_var.apply(lambda x: np.random.random() if x == '?' else float(x)).astype('float32')
df['alchemy_category_score'] = df_var_temp


# num word in url -- discrete 0-25 to custom binning from looking at the histogram
df_var = data['numwords_in_url']
bins = [0, 6, 8, 13, 25]
df_var_temp = pd.cut(x=df_var, bins=bins, right=True, labels=['num_words_url_bin_0', 'num_words_url_bin_1', 'num_words_url_bin_2', 'num_words_url_bin_3'])
dummies = pd.get_dummies(df_var_temp)
df = pd.concat([df, dummies], axis=1)


# parameterized_link_ratio -- leaving as continuous, right-half gaussian
df['parameterized_link_ratio'] = data['parametrizedLinkRatio']

# spelling errors ratio -- leaving as continuous
df['spelling_errors_ratio'] = data['spelling_errors_ratio']

# embed_ratio -- bimodal continuous binned into 2 bins
df_var = pd.DataFrame(data['embed_ratio'])
df_var = df_var['embed_ratio'].apply(lambda x: 1 if x > -1 else 0)
dummies = pd.get_dummies(df_var)
rename = {0: 'embed_ratio_0', 1: 'embed_ratio_1'}
dummies = dummies.rename(columns=rename)
df = pd.concat([df, dummies], axis=1)


# html_ratio -- leaving continuous
df['html_ratio'] = data['html_ratio']

# lengthy_link_domain
df_var = pd.get_dummies(data['lengthyLinkDomain'])
rename = {0: 'lengthy_link_domain_0', 1: 'lengthy_link_domain_1'}
df_var = df_var.rename(columns=rename)
df = pd.concat([df, df_var], axis=1)

df['labels'] = data['label']


In [16]:
train, val = train_test_split(df, test_size=0.5, train_size=0.5, random_state=234)
val, test = train_test_split(val, test_size=0.5, train_size=0.5, random_state= 675)
train_labels = train['labels']
train = train.drop(['labels'], axis=1, inplace=False)
val_labels = val['labels']
val = val.drop(['labels'], axis=1, inplace=False)
test_labels = test['labels']
test = test.drop(['labels'], axis=1, inplace=False)

## Last Time : logistic regression with numericla features : AUC=0.71

In [17]:
model = LogisticRegression()
model.fit(train, train_labels)
preds = model.predict_proba(val)[:,1]
score = roc_auc_score(val_labels, preds)
score

0.714056382787033

# 1 : Textual feature with Count Vectorizer (Bag of Words)

- min_df = minimum freuencey cut-off
- max_features = take the top 1000 most common feature
- strip_accents = to handle non english letters
- ngram_range = we are doing bag of word features here

In [18]:
# TFIDF add in the text features with tfidf
unigram_dtm = CountVectorizer(min_df= 10,  max_features= 1000, strip_accents= "unicode",
                          ngram_range=(1, 1))
unigram_dtm.fit(data["boilerplate"])
data_text = unigram_dtm.transform(data["boilerplate"])
train_text, val_text = train_test_split(data_text, test_size=0.5, train_size=0.5, random_state=234)
val_text, test_text = train_test_split(val_text, test_size=0.5, train_size=0.5, random_state= 675)

In [19]:
np.random.choice(unigram_dtm.get_feature_names(), 10)

array(['speed', 'also', 'area', 'she', 'she', 'users', 'si', 'good',
       'cells', 'small'], dtype='<U11')

In [20]:
xx = train_text.toarray()
print (xx.shape)
xx

(3697, 1000)


array([[ 2,  2,  0, ...,  2,  3,  0],
       [ 0,  0,  0, ..., 30, 33,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  2,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  7,  1, ...,  0,  0,  0]])

In [94]:
train.shape

(3697, 28)

In [95]:
train_text.toarray().shape

(3697, 1000)

In [96]:
train_with_text = pd.concat([train.reset_index(drop = True), pd.DataFrame(train_text.toarray())], axis=1)
val_with_text = pd.concat([val.reset_index(drop = True), pd.DataFrame(val_text.toarray())], axis=1)
train_with_text.head()

,alchemy_cat_?,arts_entertainment,business,computer_internet,culture_politics,gaming,health,law_crime,recreation,religion,...,990,991,992,993,994,995,996,997,998,999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,3,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,30,33,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,7,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [97]:
model = LogisticRegression()
model.fit(train_with_text, train_labels)
preds = model.predict_proba(val_with_text)[:,1]
score = roc_auc_score(val_labels, preds)
score

0.7832290811061267

## 2. Textual Features with Count Vectorizer (Bi-Gram)
- ngram_range = (2,2) now so we only bi-gram

In [98]:
# TFIDF add in the text features with tfidf
bigram_dtm = CountVectorizer(min_df= 10,  max_features= 1000, strip_accents= "unicode",
                          ngram_range=(2, 2))
bigram_dtm.fit(data["boilerplate"])
data_text = bigram_dtm.transform(data["boilerplate"])
train_text, val_text = train_test_split(data_text, test_size=0.5, train_size=0.5, random_state=234)
val_text, test_text = train_test_split(val_text, test_size=0.5, train_size=0.5, random_state= 675)

train_with_text = pd.concat([train.reset_index(drop = True), pd.DataFrame(train_text.toarray())], axis=1)
val_with_text = pd.concat([val.reset_index(drop = True), pd.DataFrame(val_text.toarray())], axis=1)
train_with_text.head()

,alchemy_cat_?,arts_entertainment,business,computer_internet,culture_politics,gaming,health,law_crime,recreation,religion,...,990,991,992,993,994,995,996,997,998,999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,4,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
np.random.choice(bigram_dtm.get_feature_names(), 10)

array(['http www', 'vegetable oil', 'at all', 'half of', 'what to',
       'people who', 'you want', 're not', 'to find', 'would be'],
      dtype='<U20')

In [100]:
model = LogisticRegression()
model.fit(train_with_text, train_labels)
preds = model.predict_proba(val_with_text)[:,1]
score = roc_auc_score(val_labels, preds)
score

0.7643770958675868

## 3. Textual Features with tfidf

In [101]:
# TFIDF add in the text features with tfidf
idf_dtm = TfidfVectorizer(min_df= 10,  max_features= 1000, strip_accents= "unicode", ngram_range=(1, 2))
idf_dtm.fit(data["boilerplate"])
data_text = idf_dtm.transform(data["boilerplate"])
train_text, val_text = train_test_split(data_text, test_size=0.5, train_size=0.5, random_state=234)
val_text, test_text = train_test_split(val_text, test_size=0.5, train_size=0.5, random_state= 675)

train_with_text = pd.concat([train.reset_index(drop = True), pd.DataFrame(train_text.toarray())], axis=1)
val_with_text = pd.concat([val.reset_index(drop = True), pd.DataFrame(val_text.toarray())], axis=1)
train_with_text.head()

,alchemy_cat_?,arts_entertainment,business,computer_internet,culture_politics,gaming,health,law_crime,recreation,religion,...,990,991,992,993,994,995,996,997,998,999
0,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.036841,0.0,0.0,0.0
1,1,0,0,0,0,0,0,0,0,0,...,0.006448,0.005204,0.0,0.000000,0.0,0.025413,0.102643,0.0,0.0,0.0
2,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.115260,0.0,0.073187,0.0,0.000000,0.034443,0.0,0.0,0.0
4,1,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0


In [102]:
np.random.choice(idf_dtm.get_feature_names(), 10)

array(['call', 'have', 'can be', 'real', 'with your', 'lemon', 'homemade',
       'to', 'need', 'better'], dtype='<U13')

In [103]:
model = LogisticRegression()
model.fit(train_with_text, train_labels)
preds = model.predict_proba(val_with_text)[:,1]
score = roc_auc_score(val_labels, preds)
score

0.8637525185512802